# Convergence of training and test error

According to statistical learning theory, training and test errors converge when the number of training samples grows for all **reasonable machine learning methods**. However, the convergence speeds really depends on many factors. The following notebook highlights the differences and causes behind them.    

In [1]:
import numpy as np
import numpy.random as random
import pandas as pd

from pandas import Series
from pandas import DataFrame
from typing import List

# Local imports
from common import *
from convenience import *

## I. Data generation functions 

Code for generating data according to logistic regression model:

* The binary data matrix X is generated randomly.
* The target vector y is generated according to logit model.

We generate two types of data matrices:

* compact matrices where all features are needed for prediction
* matrices with extra columns that do not contain any information for prediction

**Random functions** 

In [2]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def logit(x:Series, w:Series):
    """
    Labels output according to logit model pr[y=1|sigmoid(w*x)]
    
    You can omit trailing zeroes in w by specifying only first non-zero coefficients  
    """
    return random.rand() <= sigmoid(np.dot(x.iloc[0:len(w)] - 0.5, w))

**Data generation**

In [3]:
def data_sampler(n:int, k:int,  f:callable) -> DataFrame:
    """
    Data generator that generates n x k feature matrix and a target vector
    
    Returns a data frame with columns x_1, ..., x_k, y where y is computed
    using randomised labelling function f.
    """
    
    columns = ['x_{}'.format(num) for num in range(1, k + 1)]
    return (DataFrame(random.rand(n, k), columns = columns)
            .transform(lambda x: x >= 0.5)
            .assign(y = lambda df: df.apply(f, axis=1)))

## II. Example datasets

* The size of the dataset and nature of the labelling function determine the difficulty of the problem.
* If the target function is near-deterministic then there exists a good predictor.
* If the number of features is small then it is easier to learn the target function.

###  Choosing among target functions

* Let's build a target function that outputs `True` if at least one input is `True`.
* The weights `w = [1, 1]` achieve this most of the time (the relation is not absolute).
* Larger weights `[10, 10]` increase certainty and smaller weights `[0.1, 0.1]` decrease certainty.

In [4]:
fs = lambda x: logit(x, Series([1, 1]))
X = data_sampler(1000, 2, fs)
X.head()

,x_1,x_2,y
0,False,True,False
1,True,False,True
2,False,True,False
3,True,True,False
4,True,True,True


Let see what is the fraction of `True` values for each input

In [5]:
S = X.groupby(['x_1', 'x_2']).aggregate(['count', 'sum'])
S.columns = S.columns.droplevel(0)  
S = S.assign(freq = lambda df: round(df['sum']/df['count'] * 100))
S.reset_index()

,x_1,x_2,count,sum,freq
0,False,False,234,63.0,27.0
1,False,True,258,136.0,53.0
2,True,False,243,129.0,53.0
3,True,True,265,183.0,69.0


The relation between inputs and output is not so clear:
* Let's experiment with different weights to see what happens.
* For simplicity, let's convert the previous analysis to a function.

In [6]:
def summarize(X:DataFrame) -> DataFrame:
    S = X.groupby(['x_1', 'x_2']).aggregate(['count', 'sum'])
    S.columns = S.columns.droplevel(0)  
    return (S.assign(freq = lambda df: round(df['sum']/df['count'] * 100)).
            reset_index())  
display(summarize(X))

,x_1,x_2,count,sum,freq
0,False,False,234,63.0,27.0
1,False,True,258,136.0,53.0
2,True,False,243,129.0,53.0
3,True,True,265,183.0,69.0


To estimate the effect of statistical fluctuations we do two experiments and compare results.

In [7]:
X = data_sampler(1000, 2, lambda x: logit(x, Series([10, 10])))
S1 = summarize(X)
X = data_sampler(1000, 2, lambda x: logit(x, Series([10, 10])))
S2 = summarize(X)
mdisplay([S1, S2], ['Sample 1', 'Sample 2'])

X = data_sampler(1000, 2, lambda x: logit(x, Series([0.1, 0.1])))
S1 = summarize(X)
X = data_sampler(1000, 2, lambda x: logit(x, Series([0.1, 0.1])))
S2 = summarize(X)
mdisplay([S1, S2], ['Sample 1', 'Sample 2'])

Sample 1 Sample 2 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 260 
 0.0 
 0.0 
 
 
 False 
 True 
 250 
 126.0 
 50.0 
 
 
 True 
 False 
 254 
 115.0 
 45.0 
 
 
 True 
 True 
 236 
 236.0 
 100.0 
 
 
 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 232 
 0.0 
 0.0 
 
 
 False 
 True 
 257 
 129.0 
 50.0 
 
 
 True 
 False 
 235 
 128.0 
 54.0 
 
 
 True 
 True 
 276 
 276.0 
 100.0

Sample 1 Sample 2 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 269 
 133.0 
 49.0 
 
 
 False 
 True 
 223 
 115.0 
 52.0 
 
 
 True 
 False 
 267 
 120.0 
 45.0 
 
 
 True 
 True 
 241 
 137.0 
 57.0 
 
 
 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 250 
 126.0 
 50.0 
 
 
 False 
 True 
 283 
 133.0 
 47.0 
 
 
 True 
 False 
 223 
 113.0 
 51.0 
 
 
 True 
 True 
 244 
 130.0 
 53.0

**Conclusion:** The number of samples is too small to get an accurate probability estimate.

In [8]:
X = data_sampler(100000, 2, lambda x: logit(x, Series([10, 10])))
S1 = summarize(X)
X = data_sampler(100000, 2, lambda x: logit(x, Series([10, 10])))
S2 = summarize(X)
mdisplay([S1, S2], ['Sample 1', 'Sample 2'])

X = data_sampler(100000, 2, lambda x: logit(x, Series([0.1, 0.1])))
S1 = summarize(X)
X = data_sampler(100000, 2, lambda x: logit(x, Series([0.1, 0.1])))
S2 = summarize(X)
mdisplay([S1, S2], ['Sample 1', 'Sample 2'])

Sample 1 Sample 2 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 24875 
 3.0 
 0.0 
 
 
 False 
 True 
 25192 
 12656.0 
 50.0 
 
 
 True 
 False 
 25030 
 12599.0 
 50.0 
 
 
 True 
 True 
 24903 
 24901.0 
 100.0 
 
 
 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 25051 
 1.0 
 0.0 
 
 
 False 
 True 
 25146 
 12567.0 
 50.0 
 
 
 True 
 False 
 25057 
 12563.0 
 50.0 
 
 
 True 
 True 
 24746 
 24743.0 
 100.0

Sample 1 Sample 2 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 24739 
 11710.0 
 47.0 
 
 
 False 
 True 
 24894 
 12606.0 
 51.0 
 
 
 True 
 False 
 25124 
 12517.0 
 50.0 
 
 
 True 
 True 
 25243 
 13258.0 
 53.0 
 
 
 
 
 
 x_1 
 x_2 
 count 
 sum 
 freq 
 
 
 
 
 False 
 False 
 24869 
 11925.0 
 48.0 
 
 
 False 
 True 
 25020 
 12454.0 
 50.0 
 
 
 True 
 False 
 25207 
 12486.0 
 50.0 
 
 
 True 
 True 
 24904 
 12998.0 
 52.0

* Weight vector `w = [10, 10]` creates predictable instances.
* Weight vector `w = [0.1, 0.1]` creates almost unpredictable instances.

### Choosing learning tasks to compare

* Let's experiment with predictable and almost unpredictable labelling functions.
* Let's experiment with small and big number of features.

In [9]:
sampler_11 = lambda n: data_sampler(n, 2, lambda x: logit(x, Series([10, 10])))
sampler_10 = lambda n: data_sampler(n, 2, lambda x: logit(x, Series([0.1, 0.1])))
sampler_01 = lambda n: data_sampler(n, 15, lambda x: logit(x, Series([10, 10])))
sampler_00 = lambda n: data_sampler(n, 15, lambda x: logit(x, Series([0.1, 0.1])))

## III. Learning algorithms

* Let's observe the behaviour of several algorithms.
* Majority voting is the simplest and the best classification algorithm:
  * It is optimal if the number of samples is large enough to cover each possible input.
  * It is theoretically optimal – we cannot do better without extra knowledge about the data.
* Logistic regression can work with fewer samples:
  * It makes extra assumptions about the labelling function.
  * As a result, there are fewer parameters to learn.
  * The convergence to the final classification algorithm is faster.

###  Majority voting algorithm

* Our implementation corresponds to `sklearn` prediction API:
  * constructor for fixing free hyperparameters
  * method `fit(samples, targets)` to train the model
  * method `predict(samples)` to predict labels
  * method `set_params(...)` to set hyperparameters  

In [10]:
class MajorityVoting:
    
    def __init__(self, features:List[str]=None):
        if features:
            self.features = list(features)
        else:
            self.features = None
    
    def set_params(features: List[str]) -> None:
        self.features = features
    
    def fit(self, X: DataFrame, y: Series) -> None:
        
        if self.features is None:
            self.features = list(X.columns.values)

        data = X.assign(y = y)
        pred = data.groupby(self.features).aggregate(['count', 'sum'])
        pred.columns = pred.columns.droplevel(0)
        self.pred = DataFrame({'prediction':(pred['sum']/pred['count'] >= 0.5)})
    
    def predict(self, X: DataFrame) -> np.array:
        
        return (X[self.features]
                .join(self.pred, on=self.features, how='left')['prediction']
                .fillna(True)
                .values)

**Majority voting:** example run

In [11]:
clf = MajorityVoting()
data = sampler_11(2)
features = list(data.columns.values)[0:-1]
clf.fit(data[features], data['y'])
mdisplay([data, clf.pred],['Data', 'Predictor'])

Data Predictor 
 
 
 x_1 
 x_2 
 y 
 
 
 
 
 True 
 True 
 True 
 
 
 True 
 False 
 True 
 
 
 
 
 
 prediction 
 
 
 
 
 True 
 
 
 True

**Logistic regression:** example run

In [12]:
from sklearn.linear_model import LogisticRegression
data = sampler_11(2)

clf = LogisticRegression(solver = 'lbfgs')
features = list(data.columns.values)[0:-1]
clf.fit(data[features], data['y'])
clf.predict(data[features])

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: True

## IV.  Convergence plots for accuracy

* We generate different datasets of different sizes and estimate classifiers' performance on the test set.
* To eliminate fluctuations based on test data generation, we use the same testset for all training set sizes.

### Generation of training and test data

In [ ]:
sizes = [10, 50, 100] + list(range(200, 1001, 100)) + list(range(2000, 5001, 1000))

train_00 = [sampler_00(n) for n in sizes]
features_00 = list(train_00[0].columns.values[0:-1])

train_01 = [sampler_01(n) for n in sizes]
features_01 = list(train_01[0].columns.values[0:-1])

train_10 = [sampler_10(n) for n in sizes]
features_10 = list(train_10[0].columns.values[0:-1])

train_11 = [sampler_11(n) for n in sizes]
features_11 = list(train_11[0].columns.values[0:-1])

n = 10000
test_00 = sampler_00(n)
test_01 = sampler_01(n)
test_10 = sampler_10(n)
test_11 = sampler_11(n)

### Majority Voting example  

To work out the details, it is always easier to do the analysis for one dataset.

In [ ]:
test = test_11
train = train_11[1]
clf = MajorityVoting(features_11)
clf.fit(train, train['y'])
display(clf.pred)

In [ ]:
mdisplay([train.head(), DataFrame(clf.predict(train)).head()], ['Data', 'Prediction'])

In [ ]:
test_error = sum(test['y'] != clf.predict(test)) / len(test) * 100
training_error = sum(train['y'] != clf.predict(train)) / len(train) * 100
print('Training error: {tr}% \nTest error:     {te}%'.format(tr=round(training_error), te=round(test_error)))

### Logistic regression example

In [ ]:
test = test_11
train = train_11[1]
clf = LogisticRegression(solver = 'lbfgs')
clf.fit(train[features_11], train['y'])

In [ ]:
test_error = sum(test['y'] != clf.predict(test[features_11])) / len(test) * 100
training_error = sum(train['y'] != clf.predict(train[features_11])) / len(train) * 100
print('Training error: {tr}% \nTest error:     {te}%'.format(tr=round(training_error), te=round(test_error)))

### Complete analysis

Lets generate multi-indexed key-value data frame for storing errors.

In [ ]:
df = (combine_categories({'method': ['MV', 'LR'], 'size': sizes, 'type': ['test', 'train']})
      .assign(error=np.nan, accuracy=np.nan))

error = (pd.concat([
         df.assign(source = '00', fit = 'bad',  dim = 'large'),
         df.assign(source = '01', fit = 'good', dim = 'large'),
         df.assign(source = '10', fit = 'bad',  dim = 'small'),
         df.assign(source = '11', fit = 'good', dim = 'small')])
         .set_index(['method', 'size', 'type', 'source']))[['dim', 'fit', 'error', 'accuracy']]

error = error.sort_index()
display(error.head())

Iterate over all datasets and fill the table for majority voting.

In [ ]:
for i, size in enumerate(sizes):
    test = test_00
    train = train_00[i]
    clf = MajorityVoting(features_00)
    clf.fit(train, train['y'])
    error.loc[('MV',size,'test' ,'00'), 'error'] = sum(test['y'] != clf.predict(test)) / len(test) * 100
    error.loc[('MV',size,'train','00'), 'error'] = sum(train['y'] != clf.predict(train)) / len(train) * 100
    
    test = test_01
    train = train_01[i]
    clf = MajorityVoting(features_01)
    clf.fit(train, train['y'])
    error.loc[('MV',size,'test', '01'), 'error'] = sum(test['y'] != clf.predict(test)) / len(test) * 100
    error.loc[('MV',size,'train','01'), 'error'] = sum(train['y'] != clf.predict(train)) / len(train) * 100

    test = test_10
    train = train_10[i]
    clf = MajorityVoting(features_10)
    clf.fit(train, train['y'])
    error.loc[('MV',size,'test', '10'), 'error'] = sum(test['y'] != clf.predict(test)) / len(test) * 100
    error.loc[('MV',size,'train','10'), 'error'] = sum(train['y'] != clf.predict(train)) / len(train) * 100

    test = test_11
    train = train_11[i]
    clf = MajorityVoting(features_11)
    clf.fit(train, train['y'])
    error.loc[('MV',size,'test', '11'), 'error'] = sum(test['y'] != clf.predict(test)) / len(test) * 100
    error.loc[('MV',size,'train','11'), 'error'] = sum(train['y'] != clf.predict(train)) / len(train) * 100

display(error.loc['MV', 'error'].unstack())  

In [ ]:
for i, size in enumerate(sizes):
    test = test_00
    train = train_00[i]
    features = features_00 
    clf = LogisticRegression(solver = 'lbfgs')
    clf.fit(train[features], train['y'])  
    error.loc[('LR',size,'test', '00'), 'error'] = sum(test['y'] != clf.predict(test[features])) / len(test) * 100
    error.loc[('LR',size,'train','00'), 'error'] = sum(train['y'] != clf.predict(train[features])) / len(train) * 100

    test = test_01
    train = train_01[i]
    features = features_01 
    clf = LogisticRegression(solver = 'lbfgs')
    clf.fit(train[features], train['y'])    
    error.loc[('LR',size,'test', '01'), 'error'] = sum(test['y'] != clf.predict(test[features])) / len(test) * 100
    error.loc[('LR',size,'train','01'), 'error'] = sum(train['y'] != clf.predict(train[features])) / len(train) * 100

    test = test_10
    train = train_10[i]
    features = features_10 
    clf = LogisticRegression(solver = 'lbfgs')
    clf.fit(train[features], train['y'])    
    error.loc[('LR',size,'test', '10'), 'error'] = sum(test['y'] != clf.predict(test[features])) / len(test) * 100
    error.loc[('LR',size,'train','10'), 'error'] = sum(train['y'] != clf.predict(train[features])) / len(train) * 100

    test = test_11
    train = train_11[i]
    features = features_11 
    clf = LogisticRegression(solver = 'lbfgs')
    clf.fit(train[features], train['y'])    
    error.loc[('LR',size,'test', '11'), 'error'] = sum(test['y'] != clf.predict(test[features])) / len(test) * 100
    error.loc[('LR',size,'train','11'), 'error'] = sum(train['y'] != clf.predict(train[features])) / len(train) * 100

display(error.loc['LR', 'error'].unstack())

Add accuracy for clarity and save the result.

In [ ]:
error = error.assign(error = lambda df: round(df['error'], 2), accuracy = lambda df: round(100 - df['error'], 2)).reset_index()
error.to_csv('results/convergence.csv', index = False)

### Display results

We plot two graphs, one for large-scale asymptotic behaviour and one for small-scale behaviour:

* the first shows overall convergence
* the second shows what happens in the beginning

To make our life easier, we use `ggplot` grammar of graphics for specifying what we want.

In [ ]:
from plotnine import *

In [ ]:
# Order facet dimensions
from pandas.api.types import CategoricalDtype
DimType = CategoricalDtype(['small', 'large'], ordered = True)
FitType = CategoricalDtype(['good', 'bad'], ordered = True)
df = error.assign(fit = error['fit'].astype(FitType), dim = error['dim'].astype(DimType))

In [ ]:
# Large-scale plot
p = ggplot(data = df.loc[error['size'].isin([10, 1000, 2000, 3000, 4000, 5000]),:])
p = p + geom_line(aes(x='size', y='accuracy', linetype='method', color='type'))
p = p + geom_point(aes(x='size', y='accuracy', shape='method', color='type'), fill = 'white')
p = p + facet_grid(['fit', 'dim'])
p = p + scale_shape_manual(name='Method',values=['o', 's'], labels=['LR','MV'])
p = p + scale_linetype_manual(name = 'Method', values=['-', '-.'], labels=['LR','MV']) 
p = p + scale_color_manual(name='Error type',   values=['orange', 'blue'])
p.save('convergence-large.pdf', path='results', height=6, width=12, verbose=False)
display(p)

# Small-scale plot
p = ggplot(data = df.loc[error['size'] <= 1000,:])
p = p + geom_line(aes(x='size', y='accuracy', linetype='method', color='type'))
p = p + geom_point(aes(x='size', y='accuracy', shape='method', color='type'), fill = 'white')
p = p + facet_grid(['fit', 'dim'])
p = p + scale_shape_manual(name='Method',values=['o', 's'], labels=['LR','MV'])
p = p + scale_linetype_manual(name = 'Method', values=['-', '-.'], labels=['LR','MV']) 
p = p + scale_color_manual(name='Error type',   values=['orange', 'blue'])
p.save('convergence-small.pdf', path='results', height=6, width=12, verbose=False)
display(p)

# Homework tasks

## 1.1 Classifier that minimises empirical risk (<font color='red'>1p</font>)

Given enough information about future data samples, it is possible to find a class with optimal accuracy.
The corresponding construction `MajorityVoting` was given above for the binary classification task.
* Extend the solution for multi-label classification task and apply it to the data frame `data` below.
* Predict `z` for  `x` and `y` and show the corresponding table of rules.
* What is the corresponding risk if it is defined as the probability of misclassification on `data`? 

In [ ]:
data = (DataFrame([(0, 0, 0), (0, 0, 1), (0, 0, 1), (0, 1, 2), (0, 1, 2),
                  (1, 0, 1), (1, 0, 0), (1, 0, 2), (2, 0, 1), 
                  (2, 1, 0), (2, 1, 0), (2, 1, 0), 
                  (3, 1, 1), (3, 1, 1), (3, 1, 1), (3, 1, 2)], columns = ['x', 'y', 'z'])
        .sample(frac=1).reset_index(drop = True))
display(data.head())

## 1.2 Visualise statistical fluctuations (<font color='red'>1p</font>)

The visualisations constructed above are built based on a single run where for each size we have sampled a single dataset form a data source (**distribution**). 
As such, the figure fails to capture the effect of statistical fluctuations, i.e. the effect of sampling to the results.
* Modify the code of the experiment so that 10 experiments are done for each dataset size.
* Show different scores in the graph and compute the average score line. For clarity you can do separate plots for majority voting and logistic regression.
* Justify decisions you made in the experiment design and interpret obtained results.

## 1.3 Optimise the data generation process* (<font color='red'>0.5p</font>)

The data generation procedure is slow as it does not use `numpy` matrix operations.
Fix this issue and measure the speedup.

## 1.4. Theoretical analysis of majority voting* (<font color='red'>3p</font>) 

Explain why the training accuracy is so high for majority voting. 
You can give a theoretical answer or design an experiment to answer the following questions. 
You can consider the extreme case where the features $x_i$ and labels $y$ are sampled randomly. 

* Give a rough estimate how many samples are needed to arrive to the situation where training error is roughly the same as test error. 
 
* How does the sample size depend on the number of dimensions? 
* What changes if some feature values are more probable than the others? 